# Aplicación - Knowledge Managment Officer - Proj. "Habla Tiburón" - CDF
# Jaime Florez   -   Prueba Técnica

## Initial Data Preparation & Data Exploration

In [143]:
os.getcwd()

'C:\\Galapagos\\CDFCollections240508csv1er_Ejercicio'

### READ BELOW!!!
### Please change working directory for this code to work to load the csv file.
##### I had some trouble setting up a relative path

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [144]:
csv_path = 'C:\Galapagos\CDFCollections240508csv1er_Ejercicio'
os.chdir(csv_path)
#os.path.join('..', 'Galapagos', 'CDFCollections240508csv1er_Ejercicio')
df = pd.read_csv(os.path.join(csv_path, 'CDFCollections240508csv.csv'))
df.head()

,Kingdom,Family,SciName,LatitudeDecimal,LongitudeDecimal,CollectionCode,Origin,StartDate
0,Plantae,Salviniaceae,Azolla filiculoides subsp. cristata,-0.616700,-90.350000,CDS,Native,2003-04-20 00:00:00
1,Plantae,Boraginaceae,Tournefortia pubescens,-0.496722,-90.358638,CDS,Endemic,1963-02-10 00:00:00
2,Plantae,Sapindaceae,Cardiospermum galapageium,-0.396666,-90.290833,CDS,Endemic,1963-02-10 00:00:00
3,Plantae,Euphorbiaceae,Croton scouleri var. darwinii,1.375000,-91.819400,CDS,Endemic,1963-02-16 00:00:00
4,Plantae,Boraginaceae,Tiquilia fusca,-0.582327,-90.165816,CDS,Endemic,1963-02-20 00:00:00


In [21]:
df.shape

(64765, 8)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64765 entries, 0 to 64764
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Kingdom           64765 non-null  object        
 1   Family            64765 non-null  object        
 2   SciName           64765 non-null  object        
 3   LatitudeDecimal   64765 non-null  float64       
 4   LongitudeDecimal  64765 non-null  float64       
 5   CollectionCode    64765 non-null  object        
 6   Origin            64765 non-null  object        
 7   StartDate         64765 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 4.0+ MB


In [25]:
df['StartDate'] = pd.to_datetime(df['StartDate'])

In [101]:
cc = pd.DataFrame(df.groupby('CollectionCode').size())
cc['Proportion'] = ((cc[0]*100)/df.shape[0]).round(1)
cc.columns.values[0] = "Num_Records"
cc

,Num_Records,Proportion
CollectionCode,,
CDS,33399,51.6
ICCDRS,24647,38.1
MCCDRS,5207,8.0
VCCDRS,1512,2.3


## Calculating Porcentage of Families for Each Collection

In [127]:
Fams = pd.DataFrame(df.groupby(['CollectionCode', 'Family']).size())
Fams['Porcentage'] = (Fams.groupby(level=0).apply(lambda x:  100*x / x.sum())).round(3)
Fams.columns.values[0] = "Num_Records"
Fams.reset_index(inplace = True)
Fams

C:\Users\jaime\AppData\Local\Temp\ipykernel_16236\1742461490.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  Fams['Porcentage'] = (Fams.groupby(level=0).apply(lambda x:  100*x / x.sum())).round(3)


,CollectionCode,Family,Num_Records,Porcentage
0,CDS,Acanthaceae,255,0.763
1,CDS,Acarosporaceae,35,0.105
2,CDS,Agavaceae,40,0.120
3,CDS,Agyriaceae,23,0.069
4,CDS,Ahnfeltiaceae,5,0.015
...,...,...,...,...
1032,VCCDRS,Tropiduridae,12,0.794
1033,VCCDRS,Tyrannidae,15,0.992
1034,VCCDRS,Tytonidae,15,0.992
1035,VCCDRS,Vespertilionidae,12,0.794


## Calculating Porcentage of Each Species Origin Status

In [135]:
Orig = pd.DataFrame(df.groupby(['Origin']).size())
Orig['Porcentage'] = (Orig.apply(lambda x:  100*x / x.sum())).round(2)
#  Orig['Proportion'] = ((Orig[0]*100)/df.shape[0]).round(1)  -  Diff. way
Orig.columns.values[0] = "Num_Records"
Orig.reset_index(inplace = True)
Orig

,Origin,Num_Records,Porcentage
0,Cryptogenic,2061,3.18
1,Endemic,19686,30.40
2,Introduced - eradicated,32,0.05
3,Introduced - established,17593,27.16
4,Introduced - intercepted,31,0.05
5,Migrant,52,0.08
6,Native,23889,36.89
7,No data,1384,2.14
8,Vagrant,37,0.06


## Figure for Porcentage of Families for each Collection

In [133]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import plotly.express as px

# We want a subplot for every collection
lst = list(Fams.groupby('CollectionCode'))

# Panel grid = 2 x 2
rows = 2
cols = 2
# continents are the first element in l
subplot_titles = [l[0] for l in lst]

specs = [[{'type':'domain'}] * cols] * rows

fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=subplot_titles,
        specs=specs,
        print_grid=True)

for i, l in enumerate(lst):
    row = i // cols + 1
    col = i % rows + 1
    # this is the dataframe for every Collection
    d = l[1]
    fig.add_trace(
        go.Pie(labels=d["Family"],
               values=d["Num_Records"],
               showlegend=False,
               textposition='inside',
               textinfo='label+percent'),
         row=row,
         col=col
    )
    
fig.update_layout(title="Taxonomic Families by Collection", title_x=0.5)
fig.show()

This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]
[ (2,1)  ]  [ (2,2)  ]



## Figure for Porcentage of Families for each Collection

In [137]:
fig = px.pie(Orig, values='Num_Records', names='Origin')
fig.update_layout(title="Species Origin Status", title_x=0.5)
fig.show()